In [1]:
import pandas as pd
import mysql.connector
from typing import Dict, List
from dotenv import load_dotenv # pip install python-dotenv
import os

In [2]:
# Cargar variables desde el archivo .env
load_dotenv(dotenv_path=".env")

True

In [3]:
def connect_to_database():
    """Establece la conexión con la base de datos MySQL"""
    host = os.getenv("host")
    user = os.getenv("user")
    password = os.getenv("password")
    database = os.getenv("database")

    return mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )

In [4]:
def process_excel(file_path: str) -> List[tuple]:
    """
    Procesa el archivo CSV de calificadores y retorna una lista de tuplas
    para inserción en la base de datos
    """
    # Leer el CSV
    df = pd.read_excel(file_path)
    
    # Limpiar y procesar los datos
    qualifiers = []
    for _, row in df.iterrows():
        # Verificar si tenemos un qualifierId válido
        try:
            qualifier_id = int(row['qualifierId'])
        except (ValueError, TypeError):
            continue
            
        # Verificar que tengamos un ID válido y un nombre
        if pd.notna(qualifier_id) and pd.notna(row['QUALIFIER NAME']):
            # Limpiar el nombre del calificador
            qualifier_name = str(row['QUALIFIER NAME']).strip()
            if qualifier_name:  # Solo agregar si hay un nombre válido
                qualifiers.append((
                    qualifier_id,
                    qualifier_name
                ))
    
    return qualifiers

def insert_type_qualifiers(cursor, type_qualifiers_data: List[tuple]):
    """
    Inserta o actualiza los tipos de calificadores en la base de datos
    """
    upsert_query = """
    INSERT INTO typeQualifiers 
        (typeQualifierId, qualifierName)
    VALUES 
        (%s, %s)
    ON DUPLICATE KEY UPDATE
        qualifierName = VALUES(qualifierName)
    """
    
    try:
        cursor.executemany(upsert_query, type_qualifiers_data)
        print(f"Procesados {len(type_qualifiers_data)} tipos de calificadores")
        print(f"Insertados/Actualizados: {cursor.rowcount // 2} registros")
        return True
    except Exception as e:
        print(f"Error al procesar tipos de calificadores: {str(e)}")
        raise

def verify_data(cursor):
    """
    Verifica los datos insertados en la base de datos
    """
    try:
        # Obtener el total de registros
        cursor.execute("SELECT COUNT(*) FROM typeQualifiers")
        total = cursor.fetchone()[0]
        print(f"\nTotal de tipos de calificadores en la base de datos: {total}")
        
        # Mostrar algunos ejemplos
        cursor.execute("SELECT * FROM typeQualifiers ORDER BY typeQualifierId LIMIT 5")
        print("\nEjemplos de registros insertados:")
        for record in cursor.fetchall():
            print(f"ID: {record[0]}, Nombre: {record[1]}")
            
    except Exception as e:
        print(f"Error al verificar datos: {str(e)}")


In [5]:

def main():
    connection = None
    try:
        # Establecer conexión
        print("Conectando a la base de datos...")
        connection = connect_to_database()
        cursor = connection.cursor()
        
        # Procesar el CSV
        print("Procesando archivo CSV...")
        type_qualifiers_data = process_excel('Opta_qualifiers.xlsx')
        
        if not type_qualifiers_data:
            raise ValueError("No se encontraron datos válidos para procesar en el CSV")
        
        # Insertar/Actualizar datos
        print("\nInsertando datos en la base de datos...")
        insert_type_qualifiers(cursor, type_qualifiers_data)
        
        # Verificar los datos insertados
        verify_data(cursor)
        
        # Confirmar cambios
        connection.commit()
        print("\nTodos los cambios fueron confirmados exitosamente")
        
    except Exception as error:
        print(f"\nError: {error}")
        if connection and connection.is_connected():
            connection.rollback()
            print("Se realizó rollback de los cambios")
        
    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print("Conexión cerrada")


In [6]:
if __name__ == "__main__":
    main()

Conectando a la base de datos...
Procesando archivo CSV...

Insertando datos en la base de datos...
Procesados 387 tipos de calificadores
Insertados/Actualizados: 194 registros

Total de tipos de calificadores en la base de datos: 386

Ejemplos de registros insertados:
ID: 1, Nombre: Long ball
ID: 2, Nombre: Cross
ID: 3, Nombre: Head pass
ID: 4, Nombre: Through ball
ID: 5, Nombre: Free kick taken

Todos los cambios fueron confirmados exitosamente
Conexión cerrada
